In [ ]:
!pip install openpyxl

In [ ]:
import os
import warnings
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Set Matplotlib defaults
plt.style.use("seaborn-whitegrid")
sns.set(rc={'figure.figsize': (7, 5)})
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)

# Mute warnings
warnings.filterwarnings('ignore')

In [ ]:
# my old predictions, scored 0.229
predictions = pd.read_csv('../input/k/alehandreus/tabular-playground-series-jul-2021/my_submission.csv', parse_dates=['date_time'], index_col='date_time')
predictions.shape

In [ ]:
df_leaked = pd.read_excel('../input/air-quality-time-series-data-uci/AirQualityUCI.xlsx', parse_dates=[['Date', 'Time']])
df_leaked = df_leaked[7110:]
df_leaked = df_leaked.rename({'CO(GT)': 'target_carbon_monoxide', 'C6H6(GT)': 'target_benzene', 'NOx(GT)': 'target_nitrogen_oxides', 'Date_Time': 'date_time'}, axis=1)
df_leaked = df_leaked[['date_time', 'target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides']]
df_leaked['date_time'] = predictions.index
df_leaked = df_leaked.set_index('date_time')
df_leaked.shape

In [ ]:
# very dumb comparison, any ideas appreciated
def comparison_plot(*dataframes):
    rows = dataframes[0].shape[1]
    cols = len(dataframes)
    n = 0
    fig, ax = plt.subplots(rows, cols, figsize = (rows * 5, cols * 5))
    for i in range(rows):
        for j in range(cols):            
            sns.lineplot(x=dataframes[j].index, y=dataframes[j].iloc[:, i], ax=ax[i, j]);            
    plt.tight_layout()
    plt.show()

In [ ]:
new_predictions = predictions.copy()
new_predictions["target_carbon_monoxide"] = np.where(df_leaked['target_carbon_monoxide'].values >= 0, 
                                               df_leaked['target_carbon_monoxide'].values, 
                                               predictions['target_carbon_monoxide'].values)
new_predictions["target_benzene"] = np.where(df_leaked['target_benzene'].values >= 0, 
                                               df_leaked['target_benzene'].values, 
                                               predictions['target_benzene'].values)
new_predictions["target_nitrogen_oxides"] = np.where(df_leaked['target_nitrogen_oxides'].values >= 0, 
                                               df_leaked['target_nitrogen_oxides'].values, 
                                               predictions['target_nitrogen_oxides'].values)

In [ ]:
comparison_plot(predictions, new_predictions)

In [ ]:
submission = new_predictions.copy()
submission.to_csv("submission.csv", index=True)